In [1]:
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns
import time
import numpy as np
from scipy import sparse
import pandas as pd

In [10]:
#TO READ

t = 8
i = 0

#EDGE_LIST
adj = sparse.load_npz(f"networks/adj_matrix_type_{str(t)}_run_{str(i)}.npz")
edge_list = np.column_stack((adj.row,adj.col,adj.data))
el_dataframe = pd.DataFrame(edge_list,columns = ['source','target','is_link'])


#NODE_LIST
df_nodes = pd.read_csv(f"networks/nodelist_{str(t)}_run_{str(i)}.csv")

edges = el_dataframe.to_numpy()

In [11]:
g = nx.from_pandas_edgelist(el_dataframe, create_using=nx.DiGraph())

In [12]:
node_attr = df_nodes.set_index('node_id').to_dict('index')
nx.set_node_attributes(g, node_attr)

In [33]:
def draw_graph(g):
    maj_nodelist = []
    min1_nodelist = []
    min2_nodelist = []

    for i in range(len(g.nodes)):
        print(i)
        
        group = g.nodes[i]['group']
        if group == 0:
            maj_nodelist.append(i)
        elif group == 1:
            min1_nodelist.append(i)
        elif group == 2:
            min2_nodelist.append(i)
        else:
            print("ERROR: group", group)

    subax2 = plt.subplot()
    pos = nx.circular_layout(g)
    nx.draw_networkx(g, nodelist=maj_nodelist, pos=pos, with_labels=False, node_size= 50, node_color='#B6312C')
    nx.draw_networkx(g, nodelist=min1_nodelist, pos=pos, with_labels=False, node_size= 50, node_color='#84CAE7')
    nx.draw_networkx(g, nodelist=min2_nodelist, pos=pos, with_labels=False, node_size= 50, node_color='#136F63')

    plt.show()

In [27]:
draw_graph(g)

0
1
2
3
4
5
6


KeyError: 6

In [34]:
# largest connected component
components = nx.connected_components(g)
largest_component = max(components, key=len)
H = G.subgraph(largest_component)

# compute centrality
centrality = nx.betweenness_centrality(H, k=10, endpoints=True)

# compute community structure
lpc = nx.community.label_propagation_communities(H)
#community_index = {n: i for i, com in enumerate(lpc) for n in com}

#### draw graph ####
fig, ax = plt.subplots(figsize=(20, 15))
pos = nx.spring_layout(H, k=0.15, seed=4572321)
node_color = nx.get_node_attributes(g,'group')
node_size = [v * 20000 for v in centrality.values()]
nx.draw_networkx(
    H,
    pos=pos,
    with_labels=False,
    node_color=node_color,
    node_size=node_size,
    edge_color="gainsboro",
    alpha=0.4,
)

# Resize figure for label readibility
ax.margins(0.1, 0.05)
fig.tight_layout()
plt.axis("off")
plt.show()

NetworkXNotImplemented: not implemented for directed type